In [ ]:
"""
Purpose: To check on the status of the soma finder


"""

In [1]:
import numpy as np
import datajoint as dj
import trimesh
from tqdm.notebook import tqdm
from pathlib import Path

from os import sys
sys.path.append("/meshAfterParty/")

import datajoint_utils as du
from importlib import reload

In [2]:
import minfig
du.config_celii()
du.set_minnie65_config_segmentation(minfig)
du.print_minnie65_config_paths(minfig)

#configuring will include the adapters
minnie,schema = du.configure_minnie_vm()

INFO - 2020-11-20 00:52:26,003 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2020-11-20 00:52:26,005 - settings - Setting database.user to celiib
INFO - 2020-11-20 00:52:26,006 - settings - Setting database.password to newceliipass
INFO - 2020-11-20 00:52:26,009 - settings - Setting stores to {'minnie65': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65', 'stage': '/mnt/dj-stor01/platinum/minnie65'}, 'meshes': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/meshes', 'stage': '/mnt/dj-stor01/platinum/minnie65/02/meshes'}, 'decimated_meshes': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/decimated_meshes', 'stage': '/mnt/dj-stor01/platinum/minnie65/02/decimated_meshes'}, 'skeletons': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/skeletons'}}
INFO - 2020-11-20 00:52:26,009 - settings - Setting enable_python_native_blobs to True
INFO - 2020-11-20 00:52:26,022 - connection - Connect

Current path for external_segmentation_path = /mnt/dj-stor01/platinum/minnie65/02
Current path for external_mesh_path = /mnt/dj-stor01/platinum/minnie65/02/meshes
Current path for external_decimated_mesh_path = /mnt/dj-stor01/platinum/minnie65/02/decimated_meshes
Current path for external_skeleton_path = /mnt/dj-stor01/platinum/minnie65/02/skeletons
Connecting celiib@at-database.ad.bcm.edu:3306


# Chccking the current progress of Soma Extraction

In [ ]:
dj.

In [ ]:
dj.config["display.limit"] = 50

In [84]:
len(minnie.BaylorSegmentCentroid())

91451

In [72]:
len(schema.jobs & "table_name='__baylor_segment_centroid'" & "status='reserved'" & "timestamp>'2020-11-19 14:00:00'")

1007

In [80]:
(schema.jobs & "table_name='__baylor_segment_centroid'").delete() #& "status='reserved'" & "timestamp>'2020-11-19 14:00:00'"

In [42]:
decimation_version = 0
decimation_ratio = 0.25
verts_min = 10000

key_source =  ((minnie.Decimation & f"n_vertices > {verts_min}").proj(decimation_version='version') & 
                        "decimation_version=" + str(decimation_version) &
                   f"decimation_ratio={decimation_ratio}" & minnie.SegToDecimateFromNuclei())

In [55]:
len(minnie.BaylorSegmentCentroid() & "multiplicity>0")/len(minnie.BaylorSegmentCentroid())*len(key_source)

64780.237189755964

In [50]:
total_multiplicity = (minnie.BaylorSegmentCentroid() & "multiplicity>0").fetch("multiplicity")

In [53]:
len(total_multiplicity[total_multiplicity>1])/len(total_multiplicity)

0.20647463675758349

In [54]:
np.sum(total_multiplicity)/len(minnie.BaylorSegmentCentroid())*len(key_source)

95336.19481369917

In [36]:
import neuron_visualizations as nviz
cur_mesh = du.fetch_segment_id_mesh(864691134219060901)
nviz.plot_objects(cur_mesh)

INFO - 2020-11-20 07:22:24,108 - settings - Setting enable_python_native_blobs to True
INFO - 2020-11-20 07:22:24,318 - settings - Setting enable_python_native_blobs to True
/usr/local/lib/python3.6/dist-packages/traittypes/traittypes.py:101: UserWarning: Given trait value dtype "float64" does not match required type "float64". A coerced copy has been created.
  np.dtype(self.dtype).name))


In [79]:
key_source =  ((minnie.Decimation & f"n_vertices > {verts_min}").proj(decimation_version='version') & 
                        "decimation_version=" + str(decimation_version) &
                   f"decimation_ratio={decimation_ratio}" & minnie.SegToDecimateFromNuclei())
len(key_source) - 87288

144

In [68]:
(schema.jobs & "table_name='__decimation'").delete()

In [62]:
len(minnie.SegToDecimateFromNuclei())-len(minnie.Mesh() & minnie.SegToDecimateFromNuclei())

478

In [65]:
len(minnie.Mesh() & minnie.SegToDecimateFromNuclei())

114306

In [66]:
len(minnie.Decimation())

645726

In [60]:
minnie.Mesh() & minnie.SegToDecimateFromNuclei()

segment_id segment id unique within each Segmentation,n_vertices number of vertices,n_faces number of faces,mesh in-place path to the hdf5 mesh file
864691131559524855,99,198,=BLOB=
864691131625403045,38,72,=BLOB=
864691131771477291,11,18,=BLOB=
864691131834277640,42,80,=BLOB=
864691131868306405,11,18,=BLOB=
864691131881790489,69,128,=BLOB=
864691131906133483,117,230,=BLOB=
864691131927301557,6,8,=BLOB=
864691131953361537,28,52,=BLOB=
864691131956238489,16,28,=BLOB=


In [71]:
len(minnie.BaylorSegmentCentroid())

90917

In [69]:
minnie.BaylorSegmentCentroid()

segment_id segment id unique within each Segmentation,decimation_version,decimation_ratio ratio of remaining mesh vertices/faces (which ones depends on what metric the decimation technique uses),soma_index index given to this soma to account for multiple somas in one base semgnet,centroid_x (EM voxels),centroid_y (EM voxels),centroid_z (EM voxels),n_vertices number of vertices,n_faces number of faces,soma_vertices array of vertices,soma_faces array of faces,multiplicity the number of somas found for this base segment,sdf sdf width value for the soma,max_side_ratio the maximum of the side length ratios used for check if soma,bbox_volume_ratio ratio of bbox (axis aligned) volume to mesh volume to use for check if soma,max_hole_length euclidean distance of the maximum hole size,run_time the amount of time to run (seconds)
864691134026921572,0,0.25,0,None,None,None,0,0,=BLOB=,=BLOB=,0,nan,nan,nan,nan,51.24646282196045
864691134141481226,0,0.25,0,None,None,None,0,0,=BLOB=,=BLOB=,0,nan,nan,nan,nan,5.449856758117676
864691134150137850,0,0.25,0,None,None,None,0,0,=BLOB=,=BLOB=,0,nan,nan,nan,nan,22.532501459121704
864691134167328989,0,0.25,1,234651,242738,24570,7876,15025,=BLOB=,=BLOB=,1,0.895,1.093,2.56,2341512.52,40.2511
864691134219060901,0,0.25,0,None,None,None,0,0,=BLOB=,=BLOB=,0,nan,nan,nan,nan,21.533657789230347
864691134221889045,0,0.25,1,320407,210530,26721,7667,15160,=BLOB=,=BLOB=,1,0.73,1.646,3.269,622280.987,51.5944
864691134242673681,0,0.25,0,None,None,None,0,0,=BLOB=,=BLOB=,0,nan,nan,nan,nan,17.20741558074951
864691134252908410,0,0.25,0,None,None,None,0,0,=BLOB=,=BLOB=,0,nan,nan,nan,nan,28.101099014282227
864691134295094289,0,0.25,0,None,None,None,0,0,=BLOB=,=BLOB=,0,nan,nan,nan,nan,22.975944995880127
864691134299321684,0,0.25,0,None,None,None,0,0,=BLOB=,=BLOB=,0,nan,nan,nan,nan,35.712708711624146


In [34]:
key_source = minnie.Decimation() & (minnie.BaylorSegmentCentroid() & "multiplicity>0").proj()
key_source

segment_id segment id unique within each Segmentation,version,decimation_ratio ratio of remaining mesh vertices/faces (which ones depends on what metric the decimation technique uses),n_vertices,n_faces,mesh in-place path to the hdf5 (decimated) mesh file
864691134437394197,0,0.25,12788,27917,=BLOB=
864691134455328437,0,0.25,10641,22656,=BLOB=
864691134511466596,0,0.25,10302,20737,=BLOB=
864691134577177340,0,0.25,13454,27489,=BLOB=
864691134621934751,0,0.25,13337,28887,=BLOB=
864691134690360103,0,0.25,25401,54674,=BLOB=
864691134703417700,0,0.25,10833,21883,=BLOB=
864691134748872669,0,0.25,11390,24708,=BLOB=
864691134762092352,0,0.25,11405,25806,=BLOB=
864691134765283324,0,0.25,12596,27717,=BLOB=


In [ ]:
error_key = (schema.jobs & "table_name='__baylor_segment_centroid'" & "status='error'" &
 "error_message='TypeError: only integer scalar arrays can be converted to a scalar index'").fetch("key")
# "error_message!='ValueError: zero-size array to reduction operation maximum which has no identity'")


In [ ]:
len(schema.jobs & "table_name='__baylor_segment_centroid'")#.delete()

# checking the soma extraction

In [18]:
curr_seg_id = 864691135855647534
pulled_mesh = du.fetch_segment_id_mesh(curr_seg_id)

INFO - 2020-11-20 01:01:45,383 - settings - Setting enable_python_native_blobs to True
INFO - 2020-11-20 01:01:45,594 - settings - Setting enable_python_native_blobs to True


In [7]:
undec_mesh = du.fetch_undecimated_segment_id_mesh(curr_seg_id)

INFO - 2020-11-20 00:56:29,483 - settings - Setting enable_python_native_blobs to True
INFO - 2020-11-20 00:56:29,642 - settings - Setting enable_python_native_blobs to True


In [19]:
du = reload(du)
somas = du.get_soma_mesh_list(curr_seg_id)
somas

INFO - 2020-11-20 01:01:59,424 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2020-11-20 01:01:59,424 - settings - Setting database.user to celiib
INFO - 2020-11-20 01:01:59,425 - settings - Setting database.password to newceliipass
INFO - 2020-11-20 01:01:59,428 - settings - Setting enable_python_native_blobs to True
INFO - 2020-11-20 01:01:59,620 - settings - Setting enable_python_native_blobs to True


[[<trimesh.Trimesh(vertices.shape=(38794, 3), faces.shape=(73257, 3))>],
 [203.7982],
 [0.611]]

In [20]:
nviz.plot_objects(meshes=somas[0])

/usr/local/lib/python3.6/dist-packages/traittypes/traittypes.py:101: UserWarning: Given trait value dtype "float64" does not match required type "float64". A coerced copy has been created.
  np.dtype(self.dtype).name))


In [21]:
import neuron_visualizations as nviz
nviz.plot_objects(pulled_mesh)

In [ ]:
tu.largest_hole_length(soma_info[0][0])

In [ ]:
sm = reload(sm)
soma_info = sm.extract_soma_center(curr_seg_id,
                      current_mesh_verts=pulled_mesh.vertices,
                      current_mesh_faces=pulled_mesh.faces)

In [ ]:
soma_info

In [ ]:
nviz.plot_objects(meshes=soma_info[0])